In [ ]:
# Project 3 notebook

# Ideas from John:
# Does home country advantage exist in the Olympics?
# Which countries have the highest medal rates per competitor?
# Do some countries get disqualified more often than others?
# How have Olympic performances/scores changed over time?
# etc.

# https://olympics.com/en/

In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

response = requests.get('https://en.wikipedia.org/wiki/Doping_at_the_Olympic_Games')

if response.status_code == 200:
    soup = BeautifulSoup(response.text)
    country_dopes = {}
    
    # This block sums up all mentions of each country in the "country" column of the doping scandal tables.
    # Added country names only include the part of the text in the column that is in a hyperlink
    # (e.g. "Russia (RUS)" gets added as "Russia")
    tables = soup.find_all('table', {'class': 'wikitable'})
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            if cells:
                country_cell = cells[1]
                country_hyperlink = country_cell.find('a')
                if country_hyperlink:
                    country_name = country_hyperlink.get_text(strip=True)
                    country_dopes[country_name] = country_dopes.get(country_name, 0) + 1
else:
    print("Failed to retrieve the webpage.")

# Assemble scraped data into dataframe
pd.set_option('display.max_rows', None)
country_dope_counts = pd.DataFrame(list(country_dopes.items()), columns=['Country', 'times_caught_doping'])
country_dope_counts


,Country,times_caught_doping
0,Sweden,5
1,Mongolia,4
2,Puerto Rico,2
3,United States,18
4,Netherlands,1
5,Spain,8
6,Austria,13
7,Iran,2
8,Bulgaria,10
9,Monaco,1


In [71]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

response_medals = requests.get('https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table')

if response_medals.status_code == 200:
    medal_soup = BeautifulSoup(response_medals.text)
    country_medals = {}

    main_medal_table = medal_soup.find_all('table', {'class': 'wikitable'})[12]
    medal_rows = main_medal_table.find_all('tr')
    for medal_row in medal_rows:
        medal_cells = medal_row.find_all(['td', 'th'])
        if medal_cells:
            country_cell_for_medals = medal_cells[1]
            country_hyperlink = country_cell_for_medals.find('a')
            if country_hyperlink:
                country_name_for_medals = country_hyperlink.get_text(strip=True)
                country_medals[country_name_for_medals] = {
                    'Gold': int(medal_cells[2].text),
                    'Silver': int(medal_cells[3].text),
                    'Bronze': int(medal_cells[4].text),
                    'Total': int(medal_cells[5].text)
                }
else:
    print("Failed to retrieve the webpage.")

# Merge doping data with medal data
country_medals_df = pd.DataFrame(country_medals).T.reset_index()
country_medals_df.columns = ['Country', 'Gold', 'Silver', 'Bronze', 'Total']
all_data = pd.merge(country_dope_counts, country_medals_df, on='Country', how='left')
all_data['medals_per_doper'] = all_data['Total'] / all_data['times_caught_doping']
all_data = all_data.sort_values(by='times_caught_doping', ascending=False).reset_index()
display(all_data)


,index,Country,times_caught_doping,Gold,Silver,Bronze,Total,medals_per_doper
0,28,Russia,136,194.0,165.0,186.0,545.0,4.007353
1,34,Ukraine,27,38.0,38.0,72.0,148.0,5.481481
2,38,Belarus,27,21.0,37.0,47.0,105.0,3.888889
3,43,Turkey,19,41.0,26.0,36.0,103.0,5.421053
4,3,United States,18,1174.0,952.0,833.0,2959.0,164.388889
5,53,Kazakhstan,14,15.0,25.0,39.0,79.0,5.642857
6,6,Austria,13,91.0,123.0,132.0,346.0,26.615385
7,14,Greece,11,35.0,45.0,41.0,121.0,11.000000
8,8,Bulgaria,10,55.0,90.0,85.0,230.0,23.000000
9,11,Poland,9,79.0,96.0,146.0,321.0,35.666667
